## Imports

In [37]:
import numpy as np
import open3d as o3d
from scipy.optimize import least_squares
from sklearn.cluster import DBSCAN

## Load point cloud

In [38]:
pcd = o3d.io.read_point_cloud("filtered2.pcd")
downpcd = pcd.voxel_down_sample(voxel_size=0.005)
points = np.asarray(downpcd.points)
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=30))
normals = np.asarray(downpcd.normals)

## RANSAC Cone

In [39]:
def fit_cone(points):
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize the direction vector
        
        # Vector from the cone vertex to the point
        vector_to_point = points - np.array([x0, y0, z0])
        
        # Projection of the vector onto the cone's axis
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector
        
        # Perpendicular distance from the point to the axis of the cone
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        
        # Expected distance based on the cone's angle
        expected_distance = np.tan(theta) * projection_length
        
        # Residuals are the difference between the actual and expected distances
        return perpendicular_distance - expected_distance

    # Initial guess for the cone parameters
    x0, y0, z0 = np.mean(points, axis=0)
    direction = np.array([0, 0, 1])  # Initial guess: cone points upwards
    theta0 = np.pi / 6  # Initial guess: 30 degrees opening angle
    initial_guess = [x0, y0, z0, direction[0], direction[1], direction[2], theta0]

    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x

def ransac_cone(points, threshold, iterations):
    best_inliers = []
    best_params = None

    for _ in range(iterations):
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        params = fit_cone(sample)
        inliers = []

        for point in points:
            x0, y0, z0, a, b, c, theta = params
            direction_vector = np.array([a, b, c])
            direction_vector /= np.linalg.norm(direction_vector)
            
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length
            
            distance = np.abs(perpendicular_distance - expected_distance)
            if distance < threshold:
                inliers.append(point)

        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_params = params

    return best_params, np.array(best_inliers)

In [40]:

def fit_cone(points, theta_min, theta_max):
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize the direction vector
        
        vector_to_point = points - np.array([x0, y0, z0])
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(theta) * projection_length
        return perpendicular_distance - expected_distance

    # Initial guess for the cone parameters
    x0, y0, z0 = np.mean(points, axis=0)
    direction = np.array([0, 0, 1])  # Initial guess: cone points upwards
    theta0 = np.pi / 4  # Initial guess: 45 degrees opening angle

    # Convert theta_min and theta_max to bounds for optimization
    tan_theta_min = np.tan(theta_min)
    tan_theta_max = np.tan(theta_max)

    # Set up bounds for [x0, y0, z0, a, b, c, theta] parameters
    bounds_lower = [-np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, tan_theta_min]
    bounds_upper = [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, tan_theta_max]

    initial_guess = [x0, y0, z0, direction[0], direction[1], direction[2], theta0]
    result = least_squares(residuals, initial_guess, args=(points,), bounds=(bounds_lower, bounds_upper))
    return result.x

In [30]:
def fit_cone(points):
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize the direction vector
        
        vector_to_point = points - np.array([x0, y0, z0])
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(theta) * projection_length
        return perpendicular_distance - expected_distance

    # Initial guess for the cone parameters
    x0, y0, z0 = np.mean(points, axis=0)
    direction = np.array([0, 0, 1])  # Initial guess: cone points upwards
    theta0 = np.pi / 4  # Initial guess: 45 degrees opening angle
    initial_guess = [x0, y0, z0, direction[0], direction[1], direction[2], theta0]

    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x

def ransac_cone(points, threshold, iterations, angle_min_deg, angle_max_deg, radius_min, radius_max):
    # Convert degrees to radians for internal use
    theta_min = np.radians(angle_min_deg)
    theta_max = np.radians(angle_max_deg)

    best_inliers = []
    best_params = None

    for _ in range(iterations):
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        params = fit_cone(sample)
        x0, y0, z0, a, b, c, theta = params

        # Check if the resulting theta is within the specified bounds
        if theta_min <= theta <= theta_max:
            direction_vector = np.array([a, b, c])
            direction_vector /= np.linalg.norm(direction_vector)
            sample_center = np.mean(sample, axis=0)
            
            # Calculate the height and radius of the cone
            vector_to_sample_center = sample_center - np.array([x0, y0, z0])
            height = np.dot(vector_to_sample_center, direction_vector)
            radius = height * np.tan(theta)
            
            # Check if the radius is within the specified bounds
            if radius_min <= radius <= radius_max:
                inliers = []
                for point in points:
                    vector_to_point = point - np.array([x0, y0, z0])
                    projection_length = np.dot(vector_to_point, direction_vector)
                    projection = projection_length * direction_vector
                    perpendicular_distance = np.linalg.norm(vector_to_point - projection)
                    expected_distance = np.tan(theta) * projection_length
                    
                    distance = np.abs(perpendicular_distance - expected_distance)
                    if distance < threshold:
                        inliers.append(point)

                if len(inliers) > len(best_inliers):
                    best_inliers = inliers
                    best_params = params

    return best_params, np.array(best_inliers)

## Run RANSAC

In [35]:
theta_min = np.pi / 6  # 30 degrees
theta_max = np.pi / 3  # 60 degrees
cone_params, cone_inliers = ransac_cone(points, threshold=0.005, iterations=500, angle_min_deg=45, angle_max_deg=60, radius_min=0.0, radius_max=0.05)

## Visualization

In [36]:
cone_cloud = o3d.geometry.PointCloud()
cone_cloud.points = o3d.utility.Vector3dVector(cone_inliers)
cone_cloud.paint_uniform_color([1,0,0])

downpcd.paint_uniform_color([0,0,0])

o3d.visualization.draw_geometries([downpcd, cone_cloud])